# 读取数据集 #

举例子，我们先创建一个人工数据集，并且存储在csv文件../data/house_tiny.csv中。  
下面我们将数据集按行写入csv文件

In [3]:
import os

os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file=os.path.join('..','data','house_tiny,scv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n')#别名
    f.write('NA,Pave,127500\n')#每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要从创建的csv中加载原始数据集，我们导入pandas包并且调用read_csv函数。  
数据集有4行3列，每行描述了房间数量，巷子类型，和房屋价格

In [8]:
import pandas as pd



data=pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


# 处理缺失值 #


有2种方法，插值法和删除法，其中插值法是用一个替代值来弥补缺失值，而删除法直接忽略缺失值，这里考虑插值

通过位置索引iloc，我们将data分成inputs和outputs，前者为data前两项，后者为data最后一项，对于inputs缺失值，我们用同一列均值替换NaN项。

In [12]:
inputs,outputs=data.iloc[:,0:2],data.iloc[:,2]
inputs = inputs.fillna(inputs.mean(numeric_only=True))
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于inputs中类别值或离散值，我们将NaN视为一个类别，由于Akkey列只接受两种类型的类别值Pave和NaN，pandas可以自动将此列转换为两列Alley_Pave和Alley_nan。alley列为Pave的行会把Alley_Pave值设为1，Alley_nan值设为0，而缺少Alley列的行会正好相反

In [20]:
inputs=pd.get_dummies(inputs,dummy_na=True)
inputs*=1#get_dummies热编码生成布尔值，需要把布尔值转成int类型如下
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


# 转换为张量格式 #

现在inputs跟outputs所有条目都是数值类型，它们可以转换为张量格式。换成张量格式后可以进一步操作。

In [21]:
import torch
x,y=torch.tensor(inputs.values),torch.tensor(outputs.values)
x,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))